In [1]:
import streamlit as st
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px
import os
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, State
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.offline as plot
import requests

In [2]:
import plotly.io as pio

pio.templates.default = "seaborn"

In [3]:
def read_feather(source_file):
    feather_file_list = os.listdir(source_file)
    df = pd.DataFrame()
    for feather_files in feather_file_list:
        if feather_files.endswith(".feather"):
            strfile = source_file + feather_files
            df1 = pd.read_feather(strfile)
            df = pd.concat([df, df1], ignore_index=True, sort=False)
    return df

In [4]:

def load_data():
    username = 'syduc993'
    repository = 'Streanlit-Project'
    folder = 'Data/Tonghop'

    url = f'https://api.github.com/repos/{username}/{repository}/contents/{folder}'

    response = requests.get(url)
    contents = response.json()
    df = pd.DataFrame()
    # Loop through the contents of the folder
    for item in contents:
        # Check if the item is a file
        if item['type'] == 'file':
            # Get the name and download URL of the file
            filename = item['name']
            file_url = item['download_url']
            # Do something with the file URL (e.g. download the file)
            df1 = pd.read_feather(file_url)
            df = pd.concat([df, df1], ignore_index=True, sort=False)
    return df

In [5]:
df = load_data()

In [6]:
df.head()

,index,Date,Mã sản phẩm,Mã siêu thị,Số lượng hủy NCC,Số lượng nhập,Số lượng thực hủy,Doanh thu,Số lượng bán,Tên ST,RSM tháng 6,AM tháng 6,Miền,Tỉnh/thành,Quận/huyện,Tồn kho siêu thị,Trạng thái kinh doanh,Tên sản phẩm,Nhóm hàng
0,0,2023-06-01,1232844000391,10005,0.0,0.0,0.000,23868.0,1.326,BHX_BDU_TUY - Tân Hiệp,2978 - Chung Hữu Trí,4391 - Nguyễn Tuấn Anh,Miền Đông,Bình Dương,TX. Tân Uyên,1.000,KD bình thường,THƠM MẬT NGUYÊN VỎ,Trái cây nội CL
1,1,2023-06-01,1232854000015,10005,0.0,0.0,-5.692,477306.0,9.292,BHX_BDU_TUY - Tân Hiệp,2978 - Chung Hữu Trí,4391 - Nguyễn Tuấn Anh,Miền Đông,Bình Dương,TX. Tân Uyên,10.156,KD bình thường,TÁO GALA MINI NHẬP KHẨU,Trái cây ngoại CL
2,2,2023-06-01,1232854000019,10005,0.0,0.0,0.078,165669.0,4.046,BHX_BDU_TUY - Tân Hiệp,2978 - Chung Hữu Trí,4391 - Nguyễn Tuấn Anh,Miền Đông,Bình Dương,TX. Tân Uyên,12.588,KD bình thường,TÁO BRAEBURN MINI NHẬP KHẨU,Trái cây ngoại CL
3,3,2023-06-01,1232854000026,10005,0.0,0.0,0.050,123916.0,1.844,BHX_BDU_TUY - Tân Hiệp,2978 - Chung Hữu Trí,4391 - Nguyễn Tuấn Anh,Miền Đông,Bình Dương,TX. Tân Uyên,12.532,KD bình thường,LÊ ĐƯỜNG,Trái cây ngoại CL
4,4,2023-06-01,1232854000115,10005,0.0,0.0,2.984,0.0,0.000,BHX_BDU_TUY - Tân Hiệp,2978 - Chung Hữu Trí,4391 - Nguyễn Tuấn Anh,Miền Đông,Bình Dương,TX. Tân Uyên,2.984,KD bình thường,NHO XANH KHÔNG HẠT ÚC,Trái cây ngoại CL


In [7]:
app = JupyterDash(__name__)
product_list = df['Tên sản phẩm'].unique().tolist()
sub_group_list = df['Nhóm hàng'].unique().tolist()
store_list = df['Mã siêu thị'].unique().tolist()
app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
            sub_group_list,
            sub_group_list[0],
            id='sub_group_selected'
            )],style={'flex': '3', 'margin-right': '10px'}),
        html.Div([
            dcc.Dropdown(
            options=product_list,
            value=product_list[0],
            id='product_selected'
            )],style={'flex': '4', 'margin-right': '10px'}),
        html.Div([
            dcc.Dropdown(
            options=store_list,
            value = store_list[0],
            id='store_selected'
        )],style={'flex': '2'}),
    ], style={'display': 'flex', 'justify-content': 'flex-start', 'align-items': 'center'}),

    dcc.Graph(id='graph-with-slider'),

])

@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('store_selected', 'value'),
    Input('sub_group_selected', 'value'),
    Input('product_selected', 'value'),
    State('store_selected', 'value'),
    State('product_selected', 'value'),)
def update_figure(store_selected,sub_group_selected,product_selected,store_state,product_state):

    fig = make_subplots(rows=2, cols=2, subplot_titles=("Số lượng nhập sản phẩm", "Số lượng bán sản phẩm","Số lượng tồn sản phẩm","Số lượng hủy sản phẩm"), horizontal_spacing=0.05)

    if not store_state:
        if not product_state:
            data_sale = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)],index=['Date'],values='Số lượng bán',aggfunc=np.sum).reset_index()
            data_nhap = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)],index=['Date'],values='Số lượng nhập',aggfunc=np.sum).reset_index()
            data_stock = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)],index=['Date'],values='Tồn kho siêu thị',aggfunc=np.sum).reset_index()
            data_huy = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)],index=['Date'],values='Số lượng thực hủy',aggfunc=np.sum).reset_index()
        else:
            data_sale = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)],index=['Date'],values='Số lượng bán',aggfunc=np.sum).reset_index()
            data_nhap = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)],index=['Date'],values='Số lượng nhập',aggfunc=np.sum).reset_index()
            data_stock = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)],index=['Date'],values='Tồn kho siêu thị',aggfunc=np.sum).reset_index()
            data_huy = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)],index=['Date'],values='Số lượng thực hủy',aggfunc=np.sum).reset_index()
    else:
        if not product_state:
            data_sale = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Mã siêu thị']==store_selected)],index=['Date'],values='Số lượng bán',aggfunc=np.sum).reset_index()
            data_nhap = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Mã siêu thị']==store_selected)],index=['Date'],values='Số lượng nhập',aggfunc=np.sum).reset_index()
            data_stock = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Mã siêu thị']==store_selected)],index=['Date'],values='Tồn kho siêu thị',aggfunc=np.sum).reset_index()
            data_huy = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Mã siêu thị']==store_selected)],index=['Date'],values='Số lượng thực hủy',aggfunc=np.sum).reset_index()
        else:
            data_sale = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)&(df['Mã siêu thị']==store_selected)],index=['Date'],values='Số lượng bán',aggfunc=np.sum).reset_index()
            data_nhap = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)&(df['Mã siêu thị']==store_selected)],index=['Date'],values='Số lượng nhập',aggfunc=np.sum).reset_index()
            data_stock = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)&(df['Mã siêu thị']==store_selected)],index=['Date'],values='Tồn kho siêu thị',aggfunc=np.sum).reset_index()
            data_huy = pd.pivot_table(df[(df['Nhóm hàng']==sub_group_selected)&(df['Tên sản phẩm']==product_selected)&(df['Mã siêu thị']==store_selected)],index=['Date'],values='Số lượng thực hủy',aggfunc=np.sum).reset_index()
    if len(data_sale) == 0:
        data_sale = pd.pivot_table(df,index=['Date'],values='Số lượng bán',aggfunc=np.sum).reset_index()
    if len(data_nhap) == 0:
        data_nhap = pd.pivot_table(df,index=['Date'],values='Số lượng nhập',aggfunc=np.sum).reset_index()

    fig.add_trace(go.Scatter(x = data_nhap["Date"], y = data_nhap["Số lượng nhập"], fill='tozeroy',showlegend=False),row=1, col=1)
    fig.add_trace(go.Scatter(x = data_sale["Date"], y = data_sale["Số lượng bán"], fill='tozeroy' ,showlegend=False),row=1, col=2)
    fig.add_trace(go.Scatter(x = data_stock["Date"], y = data_stock["Tồn kho siêu thị"], fill='tozeroy',showlegend=False),row=2, col=1)
    fig.add_trace(go.Scatter(x = data_huy["Date"], y = data_huy["Số lượng thực hủy"], fill='tozeroy' ,showlegend=False),row=2, col=2)
    
    fig.update_layout(title='Biểu đồ số lượng nhập bán hủy & tồn sản phẩm')
 
  
    return fig


if __name__ == '__main__':
    app.run_server(mode='inline')


Dash is running on http://127.0.0.1:8050/



---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File c:\Users\lesyd\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\indexes\base.py:3652, in Index.get_loc(
    self=Index(['Date'], dtype='object'),
    key='Tồn kho siêu thị'
)
   3651 try:
-> 3652     return self._engine.get_loc(casted_key)
        casted_key = 'Tồn kho siêu thị'
        self = Index(['Date'], dtype='object')
   3653 except KeyError as err:

File c:\Users\lesyd\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\_libs\index.pyx:147, in pandas._libs.index.IndexEngine.get_loc()

File c:\Users\lesyd\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\_libs\index.pyx:176, in pandas._libs.index.IndexEngine.get_loc()

File pandas\_libs\hashtable_class_helper.pxi:7080, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas\_libs\hashtable_class_helper.pxi:7088, in

In [ ]:
@app.callback(
    [Output('graph-with-slider', 'figure'),
     Output("page-content", "children")],
    [Input('store_selected', 'value'),
     Input('sub_group_selected', 'value'),
     Input('product_selected', 'value'),
     Input("url", "pathname")],
    [State('store_selected', 'value')]
)


def update_graph(store_selected, sub_group_selected, product_selected, pathname, current_store):
    if pathname == "/":
        filtered_df = df[(df["Mã siêu thị"] == store_selected) & (df["Nhóm hàng"] == sub_group_selected) & (df["Tên sản phẩm"] == product_selected)]
        
        # Create figure
        fig = make_subplots(rows=1, cols=2, subplot_titles=("Column 1", "Column 2"))
        fig.add_trace(
            px.scatter(filtered_df, x="Column 1", y="Column 2").data[0],
            row=1, col=1
        )
        fig.add_trace(
            px.scatter(filtered_df, x="Column 1", y="Column 2").data[0],
            row=1, col=2
        )

        # Create page content
        page_content = html.Div([
            html.H1("Page content"),
            html.P("This is the content of the current page."),
            dcc.Graph(
                id='example-graph',
                figure={
                    'data': [
                        {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                        {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
                    ],
                    'layout': {
                        'title': 'Dash Data Visualization'
                    }
                }
            )
        ])
        
        # Return both the figure and page content
        return fig, page_content
    else:
        return dash.no_update, html.Div('404 - Page not found')